# Getting Started with Python SDK

# Definition

The [OpenAI Agents SDK](https://github.com/openai/openai-agents-python) enables you to build agentic AI apps in a lightweight, easy-to-use package. It has 3 important parts

- Agents - LLMs equipped with instructions and tools
- Handoffs - allow agents to delegate to other agents
- Guardrails - validation of inputs
- Tracing - Built-in tracking of agent runs, allowing you to view, debug and optimize your workflows
These primitives are powerful enough to express complex relationships between tools and agents, and allow you to build real-world applications

# Setup your virtual environment
# Install SDK Agents
pip install openai-agents

 ----ensure you set the OPENAI_API_KEY environment variable---

# Creating the first agent

In [17]:
from agents import Agent, Runner
from agents import GuardrailFunctionOutput, Agent, Runner, InputGuardrail
from pydantic import BaseModel

In [23]:
agent = Agent(
    name="Math Tutor",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

result = await Runner.run(agent, "What is 3+4?")
print(result.final_output)

To solve the problem \(3 + 4\), follow these steps:

1. **Understand the Operation**: Addition is a basic arithmetic operation used to calculate the total or sum of two or more numbers.

2. **Identify the Numbers**: Here, we have the numbers 3 and 4 that need to be added.

3. **Perform the Addition**:
   - Start with 3.
   - Add 4 to it.

   Visualizing or using physical objects can help: Imagine you have 3 apples and you get 4 more apples. Counting all the apples gives you a total.

4. **Calculate the Sum**:
   - Starting from 3, count up: 4, 5, 6, 7.

The sum of \(3 + 4\) is \(7\).

**Example**: If you have 3 pencils and you receive 4 more pencils, you will have \(3 + 4 = 7\) pencils total.


# Add a few more agents

In [ ]:


physics_tutor_agent = Agent(
    name="Physics Tutor",
    handoff_description="Specialist agent for physics questions",
    instructions="You provide assistance with physics queries. Explain the concepts clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

# Handoffs
Handoffs let one agent delegate a conversation to another agent. This is useful when you have multiple specialized agents (e.g., a “Spanish-only” agent vs. “English-only” agent) or domain-specific agents (tech support vs. billing).


In [24]:
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[physics_tutor_agent, math_tutor_agent]
)

result = await Runner.run(triage_agent, "What is the derivative x in x^2 + 2x + 1?" "\n \n")
print(result.final_output)

To find the derivative of the expression \( x^2 + 2x + 1 \), we'll use the basic rules of differentiation.

### Step-by-Step Differentiation

1. **Identify Each Term**:
   - The expression is \( x^2 + 2x + 1 \).
   - It has three terms: \( x^2 \), \( 2x \), and \( 1 \).

2. **Differentiate Each Term Individually**:
   - **Derivative of \( x^2 \)**: 
     - Use the power rule: \( \frac{d}{dx}(x^n) = nx^{n-1} \).
     - Here, \( n = 2 \), so \( \frac{d}{dx}(x^2) = 2x^{2-1} = 2x \).

   - **Derivative of \( 2x \)**:
     - The derivative of a constant times a variable (\( cx \)) is the constant (\( c \)).
     - So, \( \frac{d}{dx}(2x) = 2 \).

   - **Derivative of \( 1 \)**:
     - The derivative of a constant is zero.
     - So, \( \frac{d}{dx}(1) = 0 \).

3. **Combine the Results**:
   - Put together the derivatives of each term: 
   - \( 2x + 2 + 0 = 2x + 2 \).

So, the derivative of \( x^2 + 2x + 1 \) is \( 2x + 2 \).

### Example

Let's apply this method to a similar function: \( y 

# Adding a Guardrail

Configurable safety checks for input and output validation

In [25]:
class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
)

async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

In [27]:
triage_agent = Agent(
    name = "Triage Agent",
    instructions = "You determine which agent to use based on user's homework question",
    handoffs = [physics_tutor_agent, math_tutor_agent],
    input_guardrails = [InputGuardrail(guardrail_function=homework_guardrail)] 
)

try:
    result = await Runner.run(triage_agent, "what is life" "\n \n")
    print(result.final_output)
except Exception as e:
    print("This is out of my scope")

This is out of my scope


# View Traces of agent to review

The Agents SDK automatically traces your agent runs, making it easy to track and debug the behavior of your agents